# Test that we can make copy-free views on an underlying storage and copy-free assignments to those views - for numpy and pytorch views using numpy storage


In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.text import * 

In [3]:
#numpy: verify that we can make copy-free views on an underlying storage and copy-free assignments to those views
bs,seq_len  = 32,10
nToks       = bs*(seq_len+1)
np_storage  = np.zeros(bs*2*(seq_len+1), dtype=np.int64)

#create views on the storage
npbatchview = np_storage[:nToks].reshape(bs,-1)
x,y         = npbatchview[:,0:seq_len], npbatchview[:,1:seq_len+1]

np.add(np_storage, 1, out=np_storage)

#assert that the modification of the storage is seen by the views
assert x.flatten().all() and y.flatten().all() 

#assert that a change to a slice in x outside the scope of y does not affect y
x[:,0] = 2
assert (x[:,0]==2).all() and (y == 1).all()   

#assert that a change to a slice in y outside the scope of x does not affect x
y[:,seq_len-1] = 3
assert (x[:,0]==2).all() and \
       (x[:,1:]==1).all() and \
       (y[:,seq_len-1] == 3).all() and \
       (y[:,seq_len-2] == 1).all()

#assert that x and y only modifed the first part of the underlying storage
np_storage_view = np_storage.reshape(2*bs,-1)
assert (np_storage_view[bs:,:]==1).all() 

#assert that modifcation to x and y appears in the underlying storage at the expected positions
assert (np_storage_view[:bs,0]        ==2).all() and \
       (np_storage_view[:bs,1:seq_len]==1).all() and \
       (np_storage_view[:bs,seq_len]  ==3).all() 

In [5]:
#xt,yt      = torch.as_tensor(batchview[:,0:seq_len], dtype=torch.long), torch.as_tensor(batchview[:,1:seq_len+1], dtype=torch.long)
#batchview  = torch.as_tensor( np_storage, dtype=torch.long ).reshape(bs,-1)
#print(xt.storage())
#print(yt.storage())
bs,seq_len  = 32,10
nToks       = bs*(seq_len+1)
np_storage  = np.zeros(bs*2*(seq_len+1), dtype=np.int64) #int64 because pytorch embedding-look-up requires it 
npbatchview = np_storage[:nToks].reshape(bs,-1)
np_x,np_y   = npbatchview[:,0:seq_len], npbatchview[:,1:seq_len+1]

#the following pt_storage uses np_storage as storage so it s is actually a view. It increases the refrencecount
#We could set np_storage=None AFTER the following statement and pt_storage would still hold on to the allocated storage
pt_storage  = torch.from_numpy( np_storage ) 


ptbatchview = pt_storage[:nToks].reshape(bs,-1)
pt_x,pt_y   = ptbatchview[:,0:seq_len], ptbatchview[:,1:seq_len+1]
assert type(ptbatchview) == torch.Tensor and type(pt_x) == torch.Tensor and type(pt_y) == torch.Tensor 

np.add(np_storage, 1, out=np_storage)

#assert that the modification of the storage is seen by the views
assert (pt_x.flatten()==1).all() and (pt_y.flatten()==1).all() 

#assert that a change to a slice in pt_x outside the scope of pt_y does not affect pt_y
pt_x[:,0] = 2
assert (pt_x[:,0]==2).all() and (pt_y == 1).all()   

#assert that a change to a slice in pt_y outside the scope of pt_x does not affect pt_x
pt_y[:,seq_len-1] = 3
assert (pt_x[:,0]==2).all() and \
       (pt_x[:,1:]==1).all() and \
       (pt_y[:,seq_len-1] == 3).all() and \
       (pt_y[:,seq_len-2] == 1).all()

#assert that a change to a slice in pt_y outside the scope of pt_x does not affect pt_x
pt_y[:,seq_len-1] = 3
assert (pt_x[:,0]==2).all() and \
       (pt_x[:,1:]==1).all() and \
       (pt_y[:,seq_len-1] == 3).all() and \
       (pt_y[:,seq_len-2] == 1).all()

#assert that x and y only modifed the first part of the underlying storage
pt_storage_view = pt_storage.reshape(2*bs,-1)
assert (pt_storage_view[bs:,:]==1).all() 

#assert that modifcation to x and y appears in the underlying pt_storage at the expected positions
assert (pt_storage_view[:bs,0]        ==2).all() and \
       (pt_storage_view[:bs,1:seq_len]==1).all() and \
       (pt_storage_view[:bs,seq_len]  ==3).all() 

#assert that x and y only modifed the first part of the underlying storage
np_storage_view = np_storage.reshape(2*bs,-1)
assert (np_storage_view[bs:,:]==1).all() 

#assert that modifcation to x and y appears in the underlying storage at the expected positions
assert (np_storage_view[:bs,0]        ==2).all() and \
       (np_storage_view[:bs,1:seq_len]==1).all() and \
       (np_storage_view[:bs,seq_len]  ==3).all() 

#assert that creating a tensor from np_x and np_y gives the same view as x and y
pt_x2,pt_y2 = torch.from_numpy(np_x), torch.from_numpy(np_y)
np_x[0,0]   = 99
np_y[-1,-1] = 999
assert pt_x[0,0] == 99 and pt_y[-1,-1]==999 and pt_x2[0,0] == 99 and pt_y2[-1,-1]==999